In [8]:
pip install HDDFeaturesX==0.1.3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement HDDFeaturesX==0.1.3 (from versions: 0.1.0, 0.1.1, 0.1.2)
ERROR: No matching distribution found for HDDFeaturesX==0.1.3

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from HDDFeaturesX import *

In [6]:
import numpy as np
import time
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd


# csv file
# data_path = 'Brain1.csv'
# data = pd.read_csv(data_path)
# data = np.array(data)

# mat file
# Load data
data_path = 'PCMAC.mat'
data_loaded = loadmat(data_path)
data_key = next(key for key in data_loaded.keys() if not key.startswith('__'))
data = data_loaded[data_key]


# Partition data into two parts
parts = partition_fold(data, 2)
ori_data = parts[0]
A = parts[1]

# Further split part A into 5
U = partition_fold(A, 5)

start_time = time.time()
# Initial feature selection on original data
fea_slt, ds_vector, fea_redun, sam_delete, ori_time = fast_feature_selection_ds(ori_data)

# Incrementally update feature selection as new data becomes available
add_data = np.array([])
in_fea_slt_fs = []
unuse_ds = []
nrf_fs = []
nrs_fs = []
time_in_ds_fs = np.array([])

for i in range(5):
    add_data = np.vstack([add_data, U[i]]) if add_data.size else U[i]
    result = incre_fea_slt_ds_filter_sam(ori_data, add_data, ds_vector, fea_slt)
    in_fea_slt_fs.append(result[0])
    unuse_ds.append(result[1])
    nrf_fs.append(result[2])
    nrs_fs.append(result[3])
    time_in_ds_fs = np.append(time_in_ds_fs, result[4])


# Calculate total execution time
end_time = time.time()
total_time = end_time - start_time

# Use selected features for training and testing
X_selected = ori_data[:, fea_slt]
y = ori_data[:, -1]
y = y.astype(int)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train KNN model
model = KNeighborsClassifier()
model.fit(X_train, y_train)

# Predict classes for test data
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Save time_in_ds_fs
results_path = 'timeInDSfs.txt'
np.savetxt(results_path, time_in_ds_fs, delimiter=',')

# Print total execution time
print(f"Total execution time: {total_time} seconds")

KeyboardInterrupt: 

In [11]:
fea_slt

[2270,
 990,
 2936,
 945,
 506,
 1393,
 2892,
 247,
 1809,
 2033,
 3258,
 2303,
 1794,
 3015,
 1787,
 1552,
 1966,
 2866,
 2546,
 170,
 1014]